### Introduction

In the following notebook, I will be cleaning a raw data file of listings data from Inside Airbnb

**Read in libraries**

In [35]:
import numpy as np
import modin.pandas as pd
import swifter 

**Set notebook preferences**

In [36]:
#Set pandas preferences
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

**Read in data**

In [37]:
#Set path to data on local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\01_Raw\SF Airbnb'

#Read in data
df = pd.read_csv(path + '/2020_0519_Aggregated_Listings.csv',dtype={'zipcode':'category'} ,parse_dates= ['host_since','last_review', 'first_review'],index_col=0)

### Data Overview

**Preview Data**

In [38]:
#Display data, print shape
print('Data shape:', df.shape)
display(df.head(3))

Data shape: (104833, 106)


,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,calendar_last_scraped,calendar_updated,cancellation_policy,city,cleaning_fee,country,country_code,description,experiences_offered,extra_people,first_review,guests_included,has_availability,host_about,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,house_rules,id,instant_bookable,interaction,is_business_travel_ready,is_location_exact,jurisdiction_names,last_review,last_scraped,latitude,license,listing_url,longitude,market,maximum_maximum_nights,maximum_minimum_nights,maximum_nights,maximum_nights_avg_ntm,medium_url,minimum_maximum_nights,minimum_minimum_nights,minimum_nights,minimum_nights_avg_ntm,monthly_price,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,notes,number_of_reviews,number_of_reviews_ltm,picture_url,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,security_deposit,smart_location,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price,xl_picture_url,zipcode
0,*Full access to patio and backyard (shared wit...,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",25,106,43,58,1.0,Real Bed,1.0,2.0,1,1,0,0,2020-04-08,3 weeks ago,moderate,San Francisco,$100.00,United States,US,New update: the house next door is under const...,none,$25.00,2009-07-23,2,t,We are a family with 2 boys born in 2009 and 2...,98%,t,1169,t,t,1.0,"San Francisco, California, United States",Holly,Duboce Triangle,https://a0.muscache.com/im/pictures/user/efdad...,90%,within an hour,2008-07-31,https://a0.muscache.com/im/pictures/user/efdad...,1.0,https://www.airbnb.com/users/show/1169,"['email', 'phone', 'facebook', 'reviews', 'kba']",* No Pets - even visiting guests for a short t...,958,t,A family of 4 lives upstairs with their dog. N...,f,t,"{""SAN FRANCISCO""}",2020-03-13,2020-04-08,37.76931,STR-0001256,https://www.airbnb.com/rooms/958,-122.43386,San Francisco,1125,1,1125,1125.0,NaN,1125,1,1,1.0,"$4,200.00","Bright, Modern Garden Unit - 1BR/1B",*Quiet cul de sac in friendly neighborhood *St...,Lower Haight,Western Addition,NaN,Due to the fact that we have children and a do...,240,56,https://a0.muscache.com/im/pictures/b7c2a199-4...,$170.00,Apartment,f,f,t,10.0,10.0,10.0,10.0,10.0,97.0,9.0,1.84,Entire home/apt,2.020041e+13,$100.00,"San Francisco, CA","Newly remodeled, modern, and bright garden uni...",NaN,CA,"San Francisco, CA, United States",New update: the house next door is under const...,NaN,*Public Transportation is 1/2 block away. *Ce...,"$1,120.00",NaN,94117
1,"Our deck, garden, gourmet kitchen and extensiv...",5,"{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",0,0,0,0,1.0,Real Bed,2.0,3.0,1,1,0,0,2020-04-09,2 weeks ago,strict_14_with_grace_period,San Francisco,$100.00,United States,US,We live in a large Victorian house on a quiet ...,none,$0.00,2009-05-03,2,t,Philip: English transplant to the Bay Area and...,81%,t,8904,t,f,2.0,"San Francisco, California, United States",Philip And Tania,Bernal Heights,https://a0.muscache.com/im/users/8904/profile_...,100%,within a day,2009-03-02,https://a0.muscache.com/im/users/8904/profile_...,2.0,https://www.airbnb.com/users/show/8904,"['email', 'phone', 'reviews', 'kba', 'work_ema...","Please respect the house, the art work, the fu...",5858,f,NaN,f,t,"{""SAN FR

**View data description**

In [39]:
#View data description
df.describe().T

,count,mean,std,min,25%,50%,75%,max
accommodates,104833.0,3.155514e+00,1.912261e+00,1.000000e+00,2.000000e+00,2.000000e+00,4.000000e+00,1.600000e+01
availability_30,104833.0,1.023421e+01,1.146587e+01,0.000000e+00,0.000000e+00,5.000000e+00,2.100000e+01,3.000000e+01
availability_365,104833.0,1.553914e+02,1.320901e+02,0.000000e+00,2.600000e+01,1.270000e+02,2.880000e+02,3.650000e+02
availability_60,104833.0,2.389618e+01,2.268816e+01,0.000000e+00,0.000000e+00,1.800000e+01,4.500000e+01,6.000000e+01
availability_90,104833.0,3.941996e+01,3.377281e+01,0.000000e+00,3.000000e+00,3.500000e+01,7.200000e+01,9.000000e+01
bathrooms,104692.0,1.384452e+00,8.995435e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.500000e+00,1.400000e+01
bedrooms,104794.0,1.340678e+00,9.370583e-01,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+01
beds,104620.0,1.744657e+00,1.192847e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+01
calculated_host_listings_count,104833.0,2.474455e+01,6.010639e+01,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+01,3.010000e+02
calculated_host_listings_count_entire_homes,104833.0,2.035331e+01,5.822700e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,3.010000e+02


## Data Cleaning

### Drop Columns

**Drop mostly homogenous/redundant columns and columns with only missing values**

In [40]:
df.head()

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,calendar_last_scraped,calendar_updated,cancellation_policy,city,cleaning_fee,country,country_code,description,experiences_offered,extra_people,first_review,guests_included,has_availability,host_about,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,house_rules,id,instant_bookable,interaction,is_business_travel_ready,is_location_exact,jurisdiction_names,last_review,last_scraped,latitude,license,listing_url,longitude,market,maximum_maximum_nights,maximum_minimum_nights,maximum_nights,maximum_nights_avg_ntm,medium_url,minimum_maximum_nights,minimum_minimum_nights,minimum_nights,minimum_nights_avg_ntm,monthly_price,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,notes,number_of_reviews,number_of_reviews_ltm,picture_url,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,security_deposit,smart_location,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price,xl_picture_url,zipcode
0,*Full access to patio and backyard (shared wit...,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",25,106,43,58,1.0,Real Bed,1.0,2.0,1,1,0,0,2020-04-08,3 weeks ago,moderate,San Francisco,$100.00,United States,US,New update: the house next door is under const...,none,$25.00,2009-07-23,2,t,We are a family with 2 boys born in 2009 and 2...,98%,t,1169,t,t,1.0,"San Francisco, California, United States",Holly,Duboce Triangle,https://a0.muscache.com/im/pictures/user/efdad...,90%,within an hour,2008-07-31,https://a0.muscache.com/im/pictures/user/efdad...,1.0,https://www.airbnb.com/users/show/1169,"['email', 'phone', 'facebook', 'reviews', 'kba']",* No Pets - even visiting guests for a short t...,958,t,A family of 4 lives upstairs with their dog. N...,f,t,"{""SAN FRANCISCO""}",2020-03-13,2020-04-08,37.76931,STR-0001256,https://www.airbnb.com/rooms/958,-122.43386,San Francisco,1125,1,1125,1125.0,NaN,1125,1,1,1.0,"$4,200.00","Bright, Modern Garden Unit - 1BR/1B",*Quiet cul de sac in friendly neighborhood *St...,Lower Haight,Western Addition,NaN,Due to the fact that we have children and a do...,240,56,https://a0.muscache.com/im/pictures/b7c2a199-4...,$170.00,Apartment,f,f,t,10.0,10.0,10.0,10.0,10.0,97.0,9.0,1.84,Entire home/apt,2.020041e+13,$100.00,"San Francisco, CA","Newly remodeled, modern, and bright garden uni...",NaN,CA,"San Francisco, CA, United States",New update: the house next door is under const...,NaN,*Public Transportation is 1/2 block away. *Ce...,"$1,120.00",NaN,94117
1,"Our deck, garden, gourmet kitchen and extensiv...",5,"{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",0,0,0,0,1.0,Real Bed,2.0,3.0,1,1,0,0,2020-04-09,2 weeks ago,strict_14_with_grace_period,San Francisco,$100.00,United States,US,We live in a large Victorian house on a quiet ...,none,$0.00,2009-05-03,2,t,Philip: English transplant to the Bay Area and...,81%,t,8904,t,f,2.0,"San Francisco, California, United States",Philip And Tania,Bernal Heights,https://a0.muscache.com/im/users/8904/profile_...,100%,within a day,2009-03-02,https://a0.muscache.com/im/users/8904/profile_...,2.0,https://www.airbnb.com/users/show/8904,"['email', 'phone', 'reviews', 'kba', 'work_ema...","Please respect the house, the art work, the fu...",5858,f,NaN,f,t,"{""SAN FR

In [41]:
#Extract cols with values with more than 1 unique value
df = df.loc[:,(df.nunique() != 1)]

In [42]:
#Drop missing columns
df.dropna(axis =1,how = 'all', inplace = True)

#Drop redundant columns
df.drop(['jurisdiction_names', 'market','state','neighbourhood'], axis = 1, inplace = True)

In [43]:
#Inspect cols with <=2 unique values
inspect = df.loc[:, (df.nunique() <=2)].columns.to_list()

#Check
display(df[inspect].head(3))

,host_has_profile_pic,host_identity_verified,host_is_superhost,instant_bookable,is_location_exact,require_guest_phone_verification,require_guest_profile_picture,requires_license
0,t,t,t,t,t,f,f,t
1,t,t,f,f,t,f,f,t
2,t,t,f,f,t,f,f,t


In [44]:
#Create dictionary for mapping
mapping = {'t':1,'f':0}

#Map 1's and 0's on t's and f's
df[inspect] = df[inspect].apply(lambda x: x.map(mapping, na_action='ignore'))

#Check
display(df[inspect].head(3))

,host_has_profile_pic,host_identity_verified,host_is_superhost,instant_bookable,is_location_exact,require_guest_phone_verification,require_guest_profile_picture,requires_license
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


**Drop columns containing url data or pertain to webscraping**

In [45]:
#Subset column headers containing 'url' or 'scrape' and store in drop
drop = list(df.filter(regex='url|scrape').columns)

#Drop drop list and check
df.drop(columns= df[drop], inplace=True)
df.head(1)

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,calendar_updated,cancellation_policy,city,cleaning_fee,description,extra_people,first_review,guests_included,host_about,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_response_rate,host_response_time,host_since,host_total_listings_count,host_verifications,house_rules,id,instant_bookable,interaction,is_location_exact,last_review,latitude,license,longitude,maximum_maximum_nights,maximum_minimum_nights,maximum_nights,maximum_nights_avg_ntm,minimum_maximum_nights,minimum_minimum_nights,minimum_nights,minimum_nights_avg_ntm,monthly_price,name,neighborhood_overview,neighbourhood_cleansed,notes,number_of_reviews,number_of_reviews_ltm,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,security_deposit,smart_location,space,square_feet,street,summary,transit,weekly_price,zipcode
0,*Full access to patio and backyard (shared wit...,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",25,106,43,58,1.0,Real Bed,1.0,2.0,1,1,0,0,3 weeks ago,moderate,San Francisco,$100.00,New update: the house next door is under const...,$25.00,2009-07-23,2,We are a family with 2 boys born in 2009 and 2...,98%,1.0,1169,1.0,1.0,1.0,"San Francisco, California, United States",Holly,Duboce Triangle,90%,within an hour,2008-07-31,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",* No Pets - even visiting guests for a short t...,958,1.0,A family of 4 lives upstairs with their dog. N...,1.0,2020-03-13,37.76931,STR-0001256,-122.43386,1125,1,1125,1125.0,1125,1,1,1.0,"$4,200.00","Bright, Modern Garden Unit - 1BR/1B",*Quiet cul de sac in friendly neighborhood *St...,Western Addition,Due to the fact that we have children and a do...,240,56,$170.00,Apartment,0.0,0.0,1.0,10.0,10.0,10.0,10.0,10.0,97.0,9.0,1.84,Entire home/apt,$100.00,"San Francisco, CA","Newly remodeled, modern, and bright garden uni...",NaN,"San Francisco, CA, United States",New update: the house next door is under const...,*Public Transportation is 1/2 block away. *Ce...,"$1,120.00",94117


## Clean up object and numeric columns

**Clean up numeric columns**

In [46]:
#Filter cols pertaining to prices and assign col names as a list to money_cols
money_cols = df.filter(regex = 'people|deposit|price|fee$|rate').columns.tolist()

#Remove $, and set type as numeric for money_cols
df[money_cols] = df[money_cols].replace('[$|,|%]','',regex = True).astype('float')

#Check
display(df[money_cols].head(3))

,cleaning_fee,extra_people,host_acceptance_rate,host_response_rate,monthly_price,price,security_deposit,weekly_price
0,100.0,25.0,98.0,90.0,4200.0,170.0,100.0,1120.0
1,100.0,0.0,81.0,100.0,5500.0,235.0,NaN,1600.0
2,50.0,12.0,86.0,100.0,1685.0,65.0,200.0,485.0


**Clean up object columns**

In [47]:
#Create list of columns to apply cleaning to
objects = df.select_dtypes('object').columns.to_list()

#Check
display(df[objects].head(3))

,access,amenities,bed_type,calendar_updated,cancellation_policy,city,description,host_about,host_location,host_name,host_neighbourhood,host_response_time,host_verifications,house_rules,interaction,license,name,neighborhood_overview,neighbourhood_cleansed,notes,property_type,room_type,smart_location,space,street,summary,transit,zipcode
0,*Full access to patio and backyard (shared wit...,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",Real Bed,3 weeks ago,moderate,San Francisco,New update: the house next door is under const...,We are a family with 2 boys born in 2009 and 2...,"San Francisco, California, United States",Holly,Duboce Triangle,within an hour,"['email', 'phone', 'facebook', 'reviews', 'kba']",* No Pets - even visiting guests for a short t...,A family of 4 lives upstairs with their dog. N...,STR-0001256,"Bright, Modern Garden Unit - 1BR/1B",*Quiet cul de sac in friendly neighborhood *St...,Western Addition,Due to the fact that we have children and a do...,Apartment,Entire home/apt,"San Francisco, CA","Newly remodeled, modern, and bright garden uni...","San Francisco, CA, United States",New update: the house next door is under const...,*Public Transportation is 1/2 block away. *Ce...,94117
1,"Our deck, garden, gourmet kitchen and extensiv...","{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",Real Bed,2 weeks ago,strict_14_with_grace_period,San Francisco,We live in a large Victorian house on a quiet ...,Philip: English transplant to the Bay Area and...,"San Francisco, California, United States",Philip And Tania,Bernal Heights,within a day,"['email', 'phone', 'reviews', 'kba', 'work_ema...","Please respect the house, the art work, the fu...",NaN,NaN,Creative Sanctuary,I love how our neighborhood feels quiet but is...,Bernal Heights,All the furniture in the house was handmade so...,Apartment,Entire home/apt,"San Francisco, CA",We live in a large Victorian house on a quiet ...,"San Francisco, CA, United States",NaN,The train is two blocks away and you can stop ...,94110
2,NaN,"{TV,Internet,Wifi,Kitchen,""Free street parking...",Real Bed,5 months ago,strict_14_with_grace_period,San Francisco,Nice and good public transportation. 7 minute...,7 minutes walk to UCSF hospital & school campu...,"San Francisco, California, United States",Aaron,Cole Valley,within an hour,"['email', 'phone', 'reviews', 'jumio', 'govern...","No party, No smoking, not for any kinds of smo...",NaN,NaN,A Friendly Room - UCSF/USF - San Francisco,"Shopping old town, restaurants, McDonald, Whol...",Haight Ashbury,Wi-Fi signal in common areas. Large eat in k...,Apartment,Private room,"San Francisco, CA","Settle down, S.F. resident, student, hospital,...","San Francisco, CA, United States",Nice and good public transportation. 7 minute...,N Juda Muni and bus stop. Street parking.,94117


In [49]:
#Remove punctuation
df[objects] = df[objects].apply(lambda x : x.str.replace('^\w\s]|(_)',' '))

#Check
display(df[objects].head())

,access,amenities,bed_type,calendar_updated,cancellation_policy,city,description,host_about,host_location,host_name,host_neighbourhood,host_response_time,host_verifications,house_rules,interaction,license,name,neighborhood_overview,neighbourhood_cleansed,notes,property_type,room_type,smart_location,space,street,summary,transit,zipcode
0,*Full access to patio and backyard (shared wit...,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",Real Bed,3 weeks ago,moderate,San Francisco,New update: the house next door is under const...,We are a family with 2 boys born in 2009 and 2...,"San Francisco, California, United States",Holly,Duboce Triangle,within an hour,"['email', 'phone', 'facebook', 'reviews', 'kba']",* No Pets - even visiting guests for a short t...,A family of 4 lives upstairs with their dog. N...,STR-0001256,"Bright, Modern Garden Unit - 1BR/1B",*Quiet cul de sac in friendly neighborhood *St...,Western Addition,Due to the fact that we have children and a do...,Apartment,Entire home/apt,"San Francisco, CA","Newly remodeled, modern, and bright garden uni...","San Francisco, CA, United States",New update: the house next door is under const...,*Public Transportation is 1/2 block away. *Ce...,94117
1,"Our deck, garden, gourmet kitchen and extensiv...","{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",Real Bed,2 weeks ago,strict 14 with grace period,San Francisco,We live in a large Victorian house on a quiet ...,Philip: English transplant to the Bay Area and...,"San Francisco, California, United States",Philip And Tania,Bernal Heights,within a day,"['email', 'phone', 'reviews', 'kba', 'work ema...","Please respect the house, the art work, the fu...",NaN,NaN,Creative Sanctuary,I love how our neighborhood feels quiet but is...,Bernal Heights,All the furniture in the house was handmade so...,Apartment,Entire home/apt,"San Francisco, CA",We live in a large Victorian house on a quiet ...,"San Francisco, CA, United States",NaN,The train is two blocks away and you can stop ...,94110
2,NaN,"{TV,Internet,Wifi,Kitchen,""Free street parking...",Real Bed,5 months ago,strict 14 with grace period,San Francisco,Nice and good public transportation. 7 minute...,7 minutes walk to UCSF hospital & school campu...,"San Francisco, California, United States",Aaron,Cole Valley,within an hour,"['email', 'phone', 'reviews', 'jumio', 'govern...","No party, No smoking, not for any kinds of smo...",NaN,NaN,A Friendly Room - UCSF/USF - San Francisco,"Shopping old town, restaurants, McDonald, Whol...",Haight Ashbury,Wi-Fi signal in common areas. Large eat in k...,Apartment,Private room,"San Francisco, CA","Settle down, S.F. resident, student, hospital,...","San Francisco, CA, United States",Nice and good public transportation. 7 minute...,N Juda Muni and bus stop. Street parking.,94117
3,NaN,"{TV,Internet,Wifi,Kitchen,""Free street parking...",Real Bed,9 months ago,strict 14 with grace period,San Francisco,Nice and good public transportation. 7 minute...,7 minutes walk to UCSF hospital & school campu...,"San Francisco, California, United States",Aaron,Cole Valley,within an hour,"['email', 'phone', 'reviews', 'jumio', 'govern...",no pet no smoke no party inside the building,NaN,NaN,Friendly Room Apt. Style -UCSF/USF - San Franc...,NaN,Haight Ashbury,Wi-Fi signal in common areas. Large eat in k...,Apartment,Private room,"San Francisco, CA","Settle down, S.F. resident, student, hospital,...","San Francisco, CA, United States",Nice and good public transportation. 7 minute...,"N Juda Muni, Bus and UCSF Shuttle. small shopp...",94117
4,Guests have access to everything listed and sh...,"{TV,Internet,Wifi,Kitchen,""Free street parking...",Real Bed,4 months ago,moderate,San Francisco,Pls email before booking. Interior featured i...,I'm an Interior Stylist living in SF. \r\n\r\n...,"San Francisco, California, United States",Rosy,Alamo Square,within a few hours,"['email', 'phone', 'reviews', 'kba']",House Manual and House Rules will be provided ...,NaN,STR-000

### Missing Data

In [50]:
#Import missing_calculator
from Missing_Stats import missing_calculator

#View missing statistics about df
display(missing_calculator(df))

,count,percentage
square_feet,103197,98.439423
monthly_price,91310,87.100436
weekly_price,91030,86.833344
host_acceptance_rate,73853,70.448237
notes,40695,38.818883
license,39561,37.737163
access,38877,37.084697
interaction,34889,33.280551
transit,32513,31.014089
house_rules,28191,26.891341


In [51]:
#Dropping square_feet, keeping the rest for now
df.drop('square_feet', axis = 1, inplace = True)

### Write to csv

In [52]:
#Print final shape of df
print('Shape of cleaned data:', df.shape)

#Set path to local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\02_Cleaned'

#Write file
df.to_csv(path + '/2020_0520_Listings_Cleaned.csv')

Shape of cleaned data: (104833, 84)
